In [1]:
# This notebook contains code for executing the tasks described (and depicted) in
# Tutorial Step 1: Oracle and Data Generation of the BFBrain documentation.

import sympy as sym
from sympy import I
from sympy.physics.quantum.dagger import Dagger

# Write a SymPy function representing the scalar potential.
def V_2HDM(phi, lam):
    Phi1 = sym.Matrix([0, phi[0]])
    Phi2 = sym.Matrix([phi[1] + I*phi[3], phi[2] + I*phi[4]])
    phi1sq = Dagger(Phi1).dot(Phi1)
    phi2sq = sym.simplify(Dagger(Phi2).dot(Phi2))
    phi12 = sym.simplify(Dagger(Phi1).dot(Phi2))
    phi21 = sym.simplify(Dagger(Phi2).dot(Phi1))

    QVec = (sym.Matrix([(phi1sq**2)/2, (phi2sq**2)/2,
                 phi1sq*phi2sq, phi12*phi21, 
                 (phi12**2 + phi21**2)/2,
                 I*(phi12**2 - phi21**2)/2,
                 phi1sq*(phi12 + phi21),
                 I*phi1sq*(phi12 - phi21),
                 phi2sq*(phi12 + phi21),
                 I*phi2sq*(phi12-phi21)])).applyfunc(sym.simplify)
    return QVec.dot(lam)

In [2]:
# Check that the SymPy function does not contain any implicit
# matrix operations, e.g. trace, that sympy.lambdify can't parse.
phisym = sym.Array(sym.symbols('phi:5', real = True))
lamsym = sym.Array(sym.symbols('lambda:10', real = True))
V_2HDM(phisym, lamsym)

lambda0*phi0**4/2 + lambda1*(phi1**2 + phi2**2 + phi3**2 + phi4**2)**2/2 + lambda2*phi0**2*(phi1**2 + phi2**2 + phi3**2 + phi4**2) + lambda3*phi0**2*(phi2**2 + phi4**2) + lambda4*phi0**2*(phi2**2 - phi4**2) - 2*lambda5*phi0**2*phi2*phi4 + 2*lambda6*phi0**3*phi2 - 2*lambda7*phi0**3*phi4 + 2*lambda8*phi0*phi2*(phi1**2 + phi2**2 + phi3**2 + phi4**2) - 2*lambda9*phi0*phi4*(phi1**2 + phi2**2 + phi3**2 + phi4**2)

In [4]:
# Initialize the DataManager object with the default oracle.
from bfbrain import DataManager

dm = DataManager.from_func(V_2HDM, 5, 10, niter = 100)

In [5]:
# Test the default oracle (this can take as long as an hour to run-- feel free to skip!)
# If our niter gives robust results, this should return 100. If it returns something
# larger, we should reinitialize dm with that being the value of niter.
dm.check_labeller(100000, niter_step = 50, count_success = 5)

recompiling vectorized_minTest...


100

In [8]:
# Advanced usage: Custom oracles.
import numpy as np

# Specify a custom oracle function, which only checks that the model is bounded-from-below for
# vev configurations where only one Higgs gets a vev.
def label_fn(func, phi_len, polar, rng, lam, **kwargs):
    # Assuming our numerical function will come from the V_2HDM function we specified earlier,
    # we specify inputs which correspond to only one of the two Higgs fields having a nonzero vev.
    input1 = np.array([1.,0.,0.,0.,0.])
    input2 = np.array([0., 0., 1., 0., 0.])
    return np.array([func(input1, x)[0] > 0 and func(input2, x)[0] > 0 for x in lam])

# Now we also specify a new function label_check, which will replace the default method 
# called by DataManager.check_labeller. This method just returns the fraction of
# the input sets of quartic coefficients (lam) that are labelled bounded-from-below.
# For our new custom oracle function, this should be about 0.25 for uniformly sampled
# sets of quartic coefficients.
def label_check(func, phi_len, polar, rng, lam, **kwargs):
    #Notice that label_check must take the same arguments as label_fn, but can return any type and may take additional keyword arguments.
    n_inputs = len(lam)
    return np.count_nonzero(label_fn(func, phi_len, polar, rng, lam)) / n_inputs


dm_custom_oracle = DataManager.from_func(V_2HDM, 5, 10, lambdify_mode = 'numpy', label_fn = label_fn, label_check = label_check)

In [9]:
# Run dm_custom_oracle.check_labeller(100000) to check that our custom oracle and oracle checker
# function are working.
dm_custom_oracle.check_labeller(100000)

0.2533